<a href="https://colab.research.google.com/github/capnmav77/11th_Metaverse/blob/main/datasetgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q langchain
!pip install -q guardrails-ai
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q huggingface_hub
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from transformers import pipeline
import os
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import HumanMessage , AIMessage , SystemMessage
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from pypdf import *
import torch

In [3]:
dataset_name = "DATASET_NAME"

In [4]:
# Load the environment variables
load_dotenv()
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")

# Create a dataset
from huggingface_hub import HfApi
from datasets import load_dataset

api = HfApi(token=huggingface_api_key)


In [ ]:
# Loading the document
loader = PyPDFDirectoryLoader("./data/")
data = loader.load()

#rint(data[0])
print(len(data))

file_name = "Extracted_data.txt"

# Open the file in write mode
with open(file_name, "w", encoding="utf-8") as file:
    for item in data:
        # Extract the "page_content" attribute from each item in the list
        page_content = item.page_content
        # Write the page_content to the file followed by a newline
        file.write(page_content + "\n")

print(f"Data has been written to {file_name}")

# Step 05: Split the Extracted Data into Text Chunks
# Split the data into text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
text_chunks = text_splitter.split_documents(data)
print("Length of the whole documentation is:", len(text_chunks))


4080
Data has been written to Extracted_data.txt
Length of the whole documentation is: 8227


In [1]:
# Prompt template
json_response_format = [
    {
        "question": "In the context of ...",
        "answer": "..."
    },
    {
        "question": "In the context of ...",
        "answer": "..."
    },
    {
        "question": "In the context of ...",
        "answer": "..."
    }
]


In [3]:
from transformers import pipeline
import json

# Save the question-answer pairs to a JSONL file
def save_to_jsonl(dataset_name, question_answer_list):
    file_name = f"Hello.jsonl"

    with open(file_name, "a", encoding="utf-8") as file:
        for qa in question_answer_list:
            entry = {
                "question": qa["question"],
                "answer": qa["answer"],
                "content": qa.get("content", ""),
            }
            file.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Data has been written to {file_name}")

# Replacing the OpenAI client instantiation with Hugging Face pipeline using "bert-large-uncased-whole-word-masking-finetuned-squad" model
qa_pipeline = pipeline(task="question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Example usage:
question = "What is the capital of France?"
context = "Paris is the capital and largest city of France."
answer = qa_pipeline(question=question, context=context)

# Save the result to a JSONL file
save_to_jsonl("qa_dataset", [{"question": question, "answer": answer["answer"], "content": context}])


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Data has been written to Hello.jsonl


In [17]:
pip install bert-extractive-summarizer


In [3]:
def split_text_into_chunks(file_path, chunk_size=1000, chunk_overlap=100):
    chunks = []

    with open(file_path, 'r', encoding='utf-8') as file:
        start = 0
        while True:
            file.seek(start)
            text = file.read(chunk_size + chunk_overlap)

            if not text:
                break

            end = start + len(text)
            chunk = text[:chunk_size]
            chunks.append(chunk)

            start = end - chunk_overlap

    return chunks

# Example usage:
file_path = "Extracted_data.txt"  # Replace with your actual file path
data_chunks = split_text_into_chunks(file_path)

print(data_chunks)



KeyboardInterrupt: 

In [ ]:
from transformers import pipeline
import json


# Function to process data chunks and generate question-answer pairs
def process_and_save_to_jsonl(data_chunks, output_file="output.jsonl"):
    # Initialize the Hugging Face question-answering pipeline
    #qa_pipeline = pipeline(task="question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

    # List to store question-answer pairs
    question_answer_list = []

    # Iterate through each data chunk
    for i, chunk in enumerate(data_chunks):
        # Define a default question template or customize as needed
        question_template = f"In the context of chunk {i + 1}, {chunk[:50]}..."

        # Use the pipeline to get the answer for the current chunk
        answer = qa_pipeline(question=question_template, context=chunk)

        # Append the question-answer pair to the list
        question_answer_list.append({
            "question": question_template,
            "answer": answer["answer"],
            "content": chunk,
        })

    # Save the question-answer pairs to a JSONL file
    save_to_jsonl(output_file, question_answer_list)


process_and_save_to_jsonl(data_chunks, output_file="output.jsonl")